## URL Encoding

20.02.14  
  
Url을 Encoding한 후, url query에 city_id를 추가하는 작업

### Library 

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
from urllib import parse

### Function

Encoder

In [2]:
def change_url(before):
    # Url parsing
    url = parse.urlparse(before)
    # query를 파싱해서 딕셔너리로 반환
    query = parse.parse_qs(url.query)
    # Encoding
    result = parse.urlencode(query, doseq=True)
    return result

Text changer

In [3]:
def change_text(before, after, data):
    data = data.apply(lambda x: str(x).replace(before, after))
    return data

### Let's Start!

데이터 불러오기

In [4]:
mbl_url = pd.read_csv('Mybucketlist_url.csv', encoding = 'utf-8')

http://인 것과 https://인 것 구분

In [5]:
mbl_url['s'] = mbl_url['before'].apply(lambda x: 1 if 'https' in x else 0)

Encoding

In [6]:
mbl_url['after'] = mbl_url['before'].apply(lambda x : change_url(x))

연결자를 +가 아닌 %20으로 문자 변경

In [7]:
mbl_url['after'] = change_text('+', '%20', mbl_url['after'])

http:s//일 경우와 http://일 경우를 구분지어 url 생성

In [8]:
for i in range(len(mbl_url)):   
    if mbl_url['s'][i] == 1:
        mbl_url['after'][i] = 'https://findmybucketlist.com/search/newresult?' + mbl_url['after'][i]
    else:
        mbl_url['after'][i] = 'http://findmybucketlist.com/search/newresult?' + mbl_url['after'][i]

cityid를 붙이기 위한 'name' 뽑아내기  

In [9]:
mbl_url['name'] = mbl_url['before'].apply(lambda x: parse.parse_qsl(parse.urlparse(x).query)[0][1])

1. parse.urlparse(x).query = url을 parsing한 후 query만 뽑아낸다.  
2. parse.parse_qsl(...)[0][1] = 뽑아낸 query를 리스트로 만들고 첫번째의 value를 뽑아낸다.  

---  

City Id

In [10]:
# 데이터 불러오기 
city = pd.read_excel('city_id.xlsx')

# id가 존재하는 것만 남기기
city = city[city['id'].notnull()]

# 중복 제거
city = city.drop_duplicates(['id'], keep = 'first').reset_index(drop = True)

만들어진 City Id와 합치기

In [11]:
mbl_url = pd.merge(mbl_url, city, how = 'left', on = 'name')

float으로 로드되었는데, 없는거 제외하고 int로 바꿔주기

In [12]:
mbl_url['id'] = mbl_url['id'].apply(lambda x: '' if str(x)=='nan' else str(int(x)))

최종 url 생성하기

In [13]:
mbl_url['result'] = list(mbl_url['after'] + '&city_id=' + mbl_url['id'])

결과 없는 것들은 지워주기

In [14]:
for i in range(len(mbl_url)):
    if mbl_url['id'][i] == '':
        mbl_url['result'][i] = ''

내보내기

In [15]:
mbl_url.to_excel('Url_change_result_.xlsx', index = False)